<a href="https://githubtocolab.com/kaust-halo/geeet/blob/master/examples/notebooks/02_demo_using_GEE_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Demo using GEE data 

This notebook is a more realistic example (but still a toy example) demonstrating the use of the PT-JPL model with real datasets from GEE. We will use NDVI data from the [MODIS/MCD43A4_006_NDVI](https://developers.google.com/earth-engine/datasets/catalog/MODIS_MCD43A4_006_NDVI) dataset and meteorological data (including net radiation) from the [ECMWF/ERA5_LAND/HOURLY](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands) dataset. 

This notebook is intended to be self-contained, i.e. all the input data will be defined here. Practically, it is more efficient to first define the input data, export it as a GEE asset, and then run the model (and export outputs) separately. 

The notebook contains some cells intended to be a template for exporting the data as an asset. Other cells contain interactive visualizations of the data using `geemap`. To run these cells, you will need to install [geemap](https://geemap.org/installation/).

In [1]:
# Use the following line to install geeet if needed:
#!pip install git+https://github.com/kaust-halo/geeet
# Use the following line to install geemap if needed:
#!pip install geemap
import ee
ee.Initialize()

## Data setup 

This section only defines the datasets, no actual GEE computations are done yet. Alternatively, skip the next two cells and load the image as an asset. 

In [ ]:
## Location and time
date_start = ee.Date('2019-06-15')
date_end = date_start.advance(1, 'day')
lonlat=[38.25, 30.25] 
pt1 = ee.Geometry.Point(lonlat[0]-0.25, lonlat[1]-0.25)
pt2 = ee.Geometry.Point(lonlat[0]+0.25, lonlat[1]-0.25)
pt3 = ee.Geometry.Point(lonlat[0]+0.25, lonlat[1]+0.25)
pt4 = ee.Geometry.Point(lonlat[0]-0.25, lonlat[1]+0.25)
region = ee.Geometry.Polygon([pt1,pt2,pt3,pt4])

## Meteorological data: ECMWF ERA5 hourly reanalysis at 0.25° resolution. 
# For PT-JPL, we need: air temperature, relative humidity, surface pressure, and net radiation
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands)
# We can define relative humidity from dewpoint temperature, air temperature, and surface pressure
# see: https://www.ecmwf.int/sites/default/files/elibrary/2016/17117-part-iv-physical-processes.pdf
# equation 7.91
# net radiation = net solar radiation + net thermal radiation
bands_to_keep = ee.List(['temperature_2m', 'dewpoint_temperature_2m', 'surface_pressure', \
    'surface_net_solar_radiation_hourly', 'surface_net_thermal_radiation_hourly'])
bands_to_rename = ee.List(['air_temperature', 'dewpoint_temperature', 'surface_pressure', \
    'short_net_radiation_hourly', 'long_net_radiation_hourly']) 

Meteo_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
    .filterDate(date_start, date_end) \
    .filterBounds(region)\
    .select(bands_to_keep, bands_to_rename)  

def prepare_ECMWF(img):
    net = img.select('short_net_radiation_hourly').add(img.select('long_net_radiation_hourly'))
    Rn = net.divide(3600).rename('net_radiation') # from J/m2 to W/m2 
    T = img.select('air_temperature')  # in Kelvin
    DT = img.select('dewpoint_temperature') # in Kelvin
    P = img.select('surface_pressure') # in Pascals
    from geeet.meteo import relative_humidity # eq 7.91 in ECMWF (2016)
    RH = relative_humidity(T, DT, P)  
    img = img.addBands(Rn).addBands(RH)
    return(img)

Meteo_collection = Meteo_collection.map(prepare_ECMWF)
# We will work with the 10AM data as an example:
Meteo_img = Meteo_collection.filter(ee.Filter.eq('hour',10)).first()

## MODIS NDVI
# We first start with an annual time series of NDVI
# so we can calculate FAPAR and FAPAR_max
NDVI_one_year = ee.ImageCollection('MODIS/MCD43A4_006_NDVI') \
                  .filterDate(date_start.advance(-1,'year'), date_start.advance(1,'year')) \
                  .filterBounds(region)\
                  .select("NDVI")
# this function calculates fapar from NDVI and adds it as a new band
from geeet.vegetation import add_fapar
# we can now calculate fapar_max for this period:
fapar_max = NDVI_one_year.map(add_fapar).select('fapar').reduce(ee.Reducer.max())
# Actual image we will use for NDVI:
NDVI = NDVI_one_year.filterDate(date_start, date_end).first().select('NDVI')

## Gather all input data into a single image:
inputs_image = Meteo_img.addBands(NDVI).addBands(fapar_max)
# PT-JPL requires day of year and time information:
d = ee.Date(inputs_image.date())
doy = d.getRelative('day', 'year').add(1)   # offset of 1 because getRelative gives 0 for doy=1
time = d.getRelative('hour', 'day').add(3)  # offset of 3 hours to obtain local time in this region.
inputs_image = inputs_image.set({'doy':doy,'time':time})
inputs_image = inputs_image.clip(region)

Use the next cell as a template to export the image as an asset (or skip this cell). This will perform the computations defined above to generate the input data.

In [ ]:
## Export the input data as an earth engine asset:
task = ee.batch.Export.image.toAsset(image=inputs_image,\
                                     description='Inputs_export_demo',\
                                     assetId='projects/geeet-public/assets/ptjpl_sample_inputs_coarse',\
                                     region=region,\
                                     scale=1000)
task.start()

If you skipped the last two cells, use the following to load the inputs asset:  

In [2]:
inputs_image = ee.Image('projects/geeet-public/assets/ptjpl_sample_inputs_coarse')
lonlat=[38.25, 30.25] 
pt1 = ee.Geometry.Point(lonlat[0]-0.25, lonlat[1]-0.25)
pt2 = ee.Geometry.Point(lonlat[0]+0.25, lonlat[1]-0.25)
pt3 = ee.Geometry.Point(lonlat[0]+0.25, lonlat[1]+0.25)
pt4 = ee.Geometry.Point(lonlat[0]-0.25, lonlat[1]+0.25)
region = ee.Geometry.Polygon([pt1,pt2,pt3,pt4])

Use the following cell to display an interactive map with the data inputs. Alternatively, you can [click here](https://code.earthengine.google.com/?scriptPath=users%2Flopezvoliver%2Fgeeet%3Aptjpl_sample_inputs_coarse) to open the code editor sample script. 

In [ ]:
# Optionally display the input data as an interactive map
# (requires geeemap)
# Alternatively, use this sample code editor script to visualize it
# https://code.earthengine.google.com/?scriptPath=users%2Flopezvoliver%2Fgeeet%3Aptjpl_sample_inputs_coarse
Map = None
try:
    import geemap
    import geemap.colormaps as cm
    ndvi_pal = cm.palettes.ndvi
    pal = cm.palettes.YlOrRd
    Map = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=12)
    Map.addLayer(inputs_image.select('relative_humidity'), {'min':0, 'max':100, 'palette':["440154","3a528b","20908d","5dc962","fde725"]}, 'Relative humidity')
    Map.addLayer(inputs_image.select('air_temperature').subtract(273.15), {'min':20, 'max':50, 'palette':pal}, 'Air temperature (C)')
    Map.addLayer(inputs_image.select('surface_pressure').divide(1000), {'min':90, 'max':110, 'palette':pal}, 'Surface pressure (KPa)')
    Map.addLayer(inputs_image.select('net_radiation'), {'min':0, 'max':500, 'palette':pal}, 'Net radiation (W/m2)')
    Map.addLayer(inputs_image.select('fapar_max'), {'min':0, 'max':1, 'palette':pal}, 'FAPAR max')
    Map.addLayer(inputs_image.select('NDVI'), {'min':0, 'max':1, 'palette':ndvi_pal}, 'NDVI')
except:
    pass

Map

## Define the PT-JPL model

Once the inputs image is prepared, the model can be define simply with one line of code:

In [3]:
from geeet.ptjpl import ptjpl_arid
ET = ptjpl_arid(inputs_image)

`ET` is now an `ee.Image` object containing all of the input bands as well as the output bands. However, note that none of the model computations have run yet - not even the computations used to define the input images! This is all done upon requesting data. This can be either by exporting the image as an asset (e.g. see template above), to drive, or by retrieving data using the `.getInfo()` method (e.g. see `01_geeet.ipynb`). The next cell adds the output bands (e.g., sensible and latent heat flux) to the map created above (if it exists - requires `geemap`). Alternatively, export the output image as an asset, or simply use this script to visualize the already processed output: https://code.earthengine.google.com/?scriptPath=users%2Flopezvoliver%2Fgeeet%3Aptjpl_sample_outputs_coarse

In [ ]:
# Adds the output data to the map above. Requires "Map" to be defined above as a geemap.Map object. 
try:
    import geemap.colormaps as cm
    pal_viridis = cm.palettes.viridis
    Map.addLayer(ET.select(['H']), {'min':0, 'max':500, 'palette':pal_viridis}, 'Sensible heat flux (W/m2)', False)
    Map.addLayer(ET.select(['G']), {'min':0, 'max':500, 'palette':pal_viridis}, 'Ground heat flux (W/m2)', False)
    Map.addLayer(ET.select(['Rn']), {'min':0, 'max':500, 'palette':pal_viridis}, 'Net radiation', False)
    Map.addLayer(ET.select(['LEs']), {'min':0, 'max':500, 'palette':pal_viridis}, 'LE soil (W/m2)', False)
    Map.addLayer(ET.select(['LEc']), {'min':0, 'max':500, 'palette':pal_viridis}, 'LE canopy (W/m2)', False)
    Map.addLayer(ET.select(['LEi']), {'min':0, 'max':500, 'palette':pal_viridis}, 'LE interception (W/m2)', False)
    Map.addLayer(ET.select(['LE']), {'min':0, 'max':500, 'palette':pal_viridis}, 'Total LE (W/m2)')
except:
    pass

Use this next cell to export the output data as an asset:

In [ ]:
## Export the output data as an earth engine asset:
task = ee.batch.Export.image.toAsset(image=ET,\
                                     description='Outputs_export_demo',\
                                     assetId='projects/geeet-public/assets/ptjpl_sample_outputs_coarse',\
                                     region=region,\
                                     scale=1000)
task.start()

## References

You can print the references for the PT-JPL model using `geeet.ptjpl.cite_all()`:

In [4]:
from geeet.ptjpl import cite_all
cite_all()

Allen, R.G., Pereira, L.S., Raes, D., Smith, M. "Crop evapotranspiration —guidelines for computing crop water requirements" (1998) FAO Irrigation and drainage paper 56. Food and Agriculture Organization, Rome, pp. 35-39. http://www.fao.org/docrep/x0490e/x0490e00.htm
Aragon, B., Houborg, R., Tu, K., Fisher, J.B., McCabe, M. "Cubesats enable high spatiotemporal retrievals of crop-water use for precision agriculture (2018)" Remote Sensing, 10 (12), art. no. 1867. http://dx.doi.org/10.3390/rs10121867
Campbell, G. S., & Norman, J. M. "Introduction to environmental biophysics (2nd ed.) (1998)"New York: Springer, pp. 168-169http://dx.doi.org/10.1007/978-1-4612-1626-1
Carlson, T.N., Capehart, W.J., Gillies, R.R. "A new look at the simplified method for remote sensing of daily evapotranspiration (1995)" Remote Sensing of Environment, 54 (2), pp. 161-167.http://dx.doi.org/10.1016/0034-4257(95)00139-R
Choudhury, B.J., Ahmed, N.U., Idso, S.B., Reginato, R.J., Daughtry, C.S.T. "Relations between ev